In [47]:
from evaluation import evaluate_base
from cnn_be import CNN_be
from cnn import CNN
import torch
from torch.nn.utils import prune

In [48]:
ckpt = "/home/chaeyoon-jang/test/fashion_mnist/ckpt2/LatentBE_model_checkpoint_epoch_193.pt"
latentbe = CNN_be()
latentbe.load_state_dict(torch.load(ckpt)['model_state_dict'])

<All keys matched successfully>

In [43]:
r_1 = latentbe.get_parameter('layer1.0.r')
s_1 = latentbe.get_parameter('layer1.0.s') + latentbe.get_parameter('layer1.0.b')
rs_1 = torch.matmul(torch.transpose(r_1, 0, 1), s_1)

r_2 = latentbe.get_parameter('layer2.0.r')
s_2 = latentbe.get_parameter('layer2.0.s') + latentbe.get_parameter('layer2.0.b')
rs_2 = torch.matmul(torch.transpose(r_2, 0, 1), s_2)

r_3 = latentbe.get_parameter('fc1.r')
s_3 = latentbe.get_parameter('fc1.s') + latentbe.get_parameter('fc1.b')
rs_3 = torch.matmul(torch.transpose(r_3, 0, 1), s_3)

r_4 = latentbe.get_parameter('fc2.r')
s_4 = latentbe.get_parameter('fc2.s') + latentbe.get_parameter('fc2.b')
rs_4 = torch.matmul(torch.transpose(r_4, 0, 1), s_4)

r_5 = latentbe.get_parameter('fc3.r')
s_5 = latentbe.get_parameter('fc3.s') + latentbe.get_parameter('fc3.b')
rs_5 = torch.matmul(torch.transpose(r_5, 0, 1), s_5)

In [44]:
destination = []
for idx, p in enumerate(latentbe.named_parameters()):
    
    if "conv.weight" in p[0]:
        
        if "layer1" in p[0]:
            destination.append(idx)
            
        elif "layer2" in p[0]:
            destination.append(idx)
            
    if "linear.weight" in p[0]:
        
        if "fc1" in p[0]:
            destination.append(idx)
            
        elif "fc2" in p[0]:
            destination.append(idx)
            
        elif "fc3" in p[0]:
            destination.append(idx)

In [45]:
group = []
for idx, p in enumerate(latentbe.parameters()):
    if idx == destination[0]:
        result = torch.mul(p.data, rs_1.expand(5,5,1,6).reshape(6,1,5,5))
        p.data = result
        
    elif idx == destination[1]:
        result = torch.mul(p.data, rs_2.expand(5,5,6,12).reshape(12,6,5,5))
        p.data = result
        
    elif idx == destination[2]:
        result = torch.mul(p.data, torch.transpose(rs_3, 0, 1))
        p.data = result
                
    elif idx == destination[3]:
        result = torch.mul(p.data, torch.transpose(rs_4, 0, 1))
        p.data = result
        
    elif idx == destination[4]:
        result = torch.mul(p.data, torch.transpose(rs_5, 0, 1)) 
        p.data = result